# Experimentovanie a vyhodnocovanie

Miroslav Čulík a Andrej Gáfrik

TODOs:
- prekopčiť veci z minulých správ do finálnej správy na overleafe https://www.overleaf.com/project/5e7101e344e1ac0001e3041b
- Experimenty
    - DM metóda: experimentovanie, nastavenie parametrov, trénovanie, testovanie, ...
    - vyhodnotenie: vyhodnotenie výsledkov, porovnanie metód (aj s publikovanými prácami)
- Zhrnutie
    - zhodnotenie toho, čo sa v rámci projektu podarilo
    - náčrt ďalšieho vhodného smerovania projektu
- Literatúra: zoznam použitej literatúry a ich citovanie v správe

- pridat vizualizacie k Decision Tree

- pridat nove modely
    - RF
    - SVR

- hyperparameter tuning
    - manual pri RF
    - gridsearch / random search
    

In [1]:
cd ..

C:\Users\Mirec\Desktop\08_fiit_ls_2019_2020\OZNAL\projekt\oznal_project


In [154]:
import seaborn as sns
import random
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.special import inv_boxcox
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from numpy.polynomial.polynomial import polyfit

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDRegressor

from src import analysis, preprocessing2, feature_selection2, metrics2

pd.set_option('max_rows', None)
pd.set_option('max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
path_housing = "data\\kc_house_data.csv"
df = pd.read_csv(path_housing, sep=",")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.00000,3,1.00000,1180,5650,1.00000,0,0,3,7,1180,0,1955,0,98178,47.51120,-122.25700,1340,5650
1,6414100192,20141209T000000,538000.00000,3,2.25000,2570,7242,2.00000,0,0,3,7,2170,400,1951,1991,98125,47.72100,-122.31900,1690,7639
2,5631500400,20150225T000000,180000.00000,2,1.00000,770,10000,1.00000,0,0,3,6,770,0,1933,0,98028,47.73790,-122.23300,2720,8062
3,2487200875,20141209T000000,604000.00000,4,3.00000,1960,5000,1.00000,0,0,5,7,1050,910,1965,0,98136,47.52080,-122.39300,1360,5000
4,1954400510,20150218T000000,510000.00000,3,2.00000,1680,8080,1.00000,0,0,3,8,1680,0,1987,0,98074,47.61680,-122.04500,1800,7503


In [4]:
X_train, X_valid, X_test, y_train, y_valid, y_test, price_lambda = preprocessing2.run_pipeline(df)

In [5]:
X_train.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode_1,zipcode_2,zipcode_3,zipcode_4,zipcode_5,zipcode_6,zipcode_7,lat,long,sqft_living15,sqft_lot15,recon_age,price_per_sqft
12496,1982201345,2,1.00000,7.38084,4.19603,1.00000,0,0,4,7,3.41952,0.00000,1944,0,0,0,0,0,0,0,1,47.66390,-122.36400,7.04752,3.99849,70,0.00248
775,4040800810,3,2.25000,8.50943,4.29079,1.00000,0,0,4,8,3.46649,31.62278,1963,0,0,0,0,0,0,1,0,47.61880,-122.11400,7.63530,4.10521,51,0.00172
17400,7525410190,3,1.75000,8.98009,4.52123,1.50000,0,0,3,8,3.66089,0.00000,1979,0,0,0,0,0,0,1,1,47.57470,-122.03500,7.85941,4.09276,35,0.00131
17038,2326059099,4,2.50000,9.14293,4.54771,2.00000,0,0,3,9,3.68127,0.00000,2001,0,0,0,0,0,1,0,0,47.72320,-122.13100,8.11672,4.09276,13,0.00157
5043,4219400290,5,2.75000,8.98009,4.32003,1.50000,0,0,3,8,3.66089,0.00000,1939,0,0,0,0,0,1,0,1,47.65520,-122.27800,7.98616,4.06938,75,0.00169


## Experimenty

Po predspracovaní dát nami vytvorenou pipeline, sme mali dáta rozdelené v pomere 70:20:10 (trénovacia : validačná : testovacia množina). Použili sme implementácie algoritmov z modulu sklearn. Najlepšie výsledky podľa metriky R$^2$ dosiahol model polynomiálnej regresie 3. stupňa s použitím selekcie čŕt RFE (0.916 na trénovacej a 0.828 na testovacej množine), druhý najlepší výsledok podľa tejto metriky dosiahol model regresného rozhodovacieho stromu(0.861 na trénovacej a 0.802 na testovacej množine). V prípade modelu jednoduchej lineárnej regresie sme vyskúšali aj 5, resp. 10 - násobnú krížovú validáciu, na porovnanie výsledkov s modelom jednoduchej lineárnej regresie trénovaného tradičným spôsobom, avšak tento prístup nám nepreukázal žiaden signifikantný rozdiel vo výsledkoch pri modeloch jednoduchej lineárnej regresie.
 

### RandomSearch pre LinearSVR

In [149]:
import random


def objective(hyperparameters, iteration, features1):

    clf = LinearSVR(**hyperparameters)
    X_my_train = X_train[features1]#.values.reshape(-1,1)
    X_my_valid = X_valid[features1]#.values.reshape(-1,1)

    clf.fit(X_my_train, np.ravel(y_train))

    y_hat = clf.predict(X_my_valid)
    mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
    rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

    r2_train = clf.score(X_my_train, y_train)
    r2_test = clf.score(X_my_valid, y_valid)
    adj_r2_train = 1 - (((1 - r2_train) * (X_my_train.shape[0] - 1)) / (X_my_train.shape[0] - X_my_train.shape[1] - 1))
    adj_r2_test = 1 - (((1 - r2_test) * (X_my_valid.shape[0] - 1)) / (X_my_valid.shape[0] - X_my_valid.shape[1] - 1))
    
    score = {
        "MSE": round(mean_squared_error, 2),
        "RMSE": round(np.sqrt(mean_squared_error),2),
        "RMSLE": round(np.sqrt(rmlse),2),
        "R2_train": round(r2_train,3),
        "R2_test": round(r2_test,3),
        "Adj_R2_train": round(adj_r2_train,3),
        "Adj_R2_test": round(adj_r2_test,3)
    }
    
    return [score, hyperparameters]


def random_search(param_grid, max_iterations, features):
    
    df_results = pd.DataFrame(columns = ['MSE', 'RMSE', 'RMSLE', 'R2_train', 'R2_test', 'Adj_R2_train', 'Adj_R2_test', 'params'],
                                  index = list(range(max_iterations)))
    
    for i in range(max_iterations):
        hyperparameters = {k: random.choice(v) for k, v in param_grid.items()}
        
        eval_results = objective(hyperparameters, i, features)
        for k,v in eval_results[0].items():
            df_results[k][i] = v
        df_results['params'][i] = eval_results[1]
    
    # Sort with best score on top
    df_results.sort_values('R2_test', ascending = False, inplace = True)
    df_results.reset_index(inplace=True)
    
    return df_results

param_grid = {
     "C": np.random.uniform(1.2, 2.0, 100),
#     "dual":[True],
#     "epsilon":np.random.uniform(0.1, 0.2, 100),
#     "fit_intercept":[True],
#     "intercept_scaling":[1],
    "loss":['epsilon_insensitive', 'squared_epsilon_insensitive'],
    "max_iter":[20000],
    "tol":np.random.uniform(1e-6, 5e-4, 100),
#     "verbose":[0]
}

random.seed(50)
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)
df_results = random_search(param_grid, 50, features1)
df_results

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\valid

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py

,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params
0,17,36349984307.61000,190656.72000,0.26000,0.75500,0.73000,0.75500,0.72900,"{'C': 1.7214568800839372, 'loss': 'epsilon_ins..."
1,24,36615664655.22000,191352.20000,0.27000,0.75000,0.72900,0.74900,0.72700,"{'C': 1.307898170860503, 'loss': 'epsilon_inse..."
2,28,36386172326.41000,190751.60000,0.27000,0.76400,0.72700,0.76300,0.72600,"{'C': 1.5344073305568182, 'loss': 'epsilon_ins..."
3,1,36377943322.08000,190730.03000,0.27000,0.76400,0.72700,0.76300,0.72600,"{'C': 1.7666620025132562, 'loss': 'squared_eps..."
4,36,36440878118.94000,190894.94000,0.27000,0.76400,0.72600,0.76400,0.72500,"{'C': 1.283269386130394, 'loss': 'epsilon_inse..."
5,29,37076331434.13000,192552.15000,0.27000,0.74300,0.72600,0.74300,0.72500,"{'C': 1.9910598311991934, 'loss': 'epsilon_ins..."
6,12,36493687564.76000,191033.21000,0.27000,0.76400,0.72500,0.76400,0.72400,"{'C': 1.4849318644445397, 'loss': 'epsilon_ins..."
7,41,36542601339.55000,191161.19000,0.27000,0.76400,0.72500,0.76400,0.72300,"{'C': 1.8890485876104075, 'loss': 'squared_eps..."
8,49,37593592052.17000,193890.67000,0.27000,0.73600,0.72300,0.73500,0.72100,"{'C': 1.9714586808164296, 'loss': 'squared_eps..."
9,21,36781777911.06000,191785.76000,0.27000,0.76500,0.72200,0.76400,0.72000,"{'C': 1.339804277498328, 'loss': 'squared_epsi..."


In [153]:
# df_results.sort_values(['R2_test'], ascending = False, inplace = True)
df_results['params'][0]

{'C': 1.7214568800839372,
 'loss': 'epsilon_insensitive',
 'max_iter': 20000,
 'tol': 0.0004935410206914095}

In [ ]:
default_params = model.get_params()

### Jednoduchá lineárna regresia - sqft_living

In [7]:
reg=linear_model.LinearRegression()
x_train=np.array(X_train['sqft_living']).reshape(-1,1)
y_train=np.array(y_train).reshape(-1,1)
reg.fit(x_train,y_train)

x_test=np.array(X_valid['sqft_living']).reshape(-1,1)
y_test=np.array(y_valid).reshape(-1,1)
pred=reg.predict(x_test)
print('linear model - sqft_living')

metrics2.evaluate(reg, x_train, y_train, x_test, y_test, pred, price_lambda)

linear model - sqft_living
Mean Squared Error 60379753642.67178
Root Mean Squared Error 245722.92047
Root Mean Squared Log Error 0.39115
R squared training 0.451
R squared testing 0.402
Adjusted-R squared training 0.451
Adjusted-R squared testing 0.402
intercept [3.78442207]
coefficient [[0.02965959]]


#### k-násobná krížová validácia pri jednoduchej lineárnej regresii

In [8]:
reg=linear_model.LinearRegression()
k = 5

X_try = pd.concat([X_train['sqft_living'], X_valid['sqft_living']])
y_try = np.concatenate((np.array(y_train).reshape(-1,1), np.array(y_valid).reshape(-1,1)), axis=0)

scores = cross_val_score(reg,  np.array(X_try).reshape(-1,1), np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())
k = 10
scores = cross_val_score(reg,  np.array(X_try).reshape(-1,1), np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())

Cross-validated scores: 0.4388515347655395
Cross-validated scores: 0.4385100197794961


### Jednoduchá lineárna regresia - grade

In [9]:
reg=linear_model.LinearRegression()
x_train=np.array(X_train['grade']).reshape(-1,1)
y_train=np.array(y_train).reshape(-1,1)
reg.fit(x_train,y_train)

x_test=np.array(X_valid['grade']).reshape(-1,1)
y_test=np.array(y_valid).reshape(-1,1)
pred=reg.predict(x_test)
print('linear model - grade')

metrics2.evaluate(reg, x_train, y_train, x_test, y_test, pred, price_lambda)

linear model - grade
Mean Squared Error 55001939903.94096
Root Mean Squared Error 234524.92384
Root Mean Squared Log Error 0.38054
R squared training 0.477
R squared testing 0.432
Adjusted-R squared training 0.477
Adjusted-R squared testing 0.432
intercept [3.93205218]
coefficient [[0.01344416]]


### Viacnásobná lineárna regresia - part1

In [10]:
features1 = feature_selection2.feature_filter(X_train, y_train, threshold=0.5)
reg=linear_model.LinearRegression()
reg.fit(X_train[features1],y_train)

pred=reg.predict(X_valid[features1])

print('multiple linear regression 1')
metrics2.evaluate(reg, X_train[features1], y_train, X_valid[features1], y_valid, pred, price_lambda)

multiple linear regression 1
Mean Squared Error 47639437017.58308
Root Mean Squared Error 218264.60322
Root Mean Squared Log Error 0.35606
R squared training 0.552
R squared testing 0.501
Adjusted-R squared training 0.551
Adjusted-R squared testing 0.501
intercept [3.96037582]
coefficient [[-0.00035035  0.01953801  0.00850982 -0.06343591  0.00938436]]


### Viacnásobná lineárna regresia - part2

In [11]:
features2 = feature_selection2.select_features_SFS(X_train, y_train, linear_model.LinearRegression)
reg=linear_model.LinearRegression()
reg.fit(X_train[features2],y_train)

pred=reg.predict(X_valid[features2])

print('multiple linear regression 2')
metrics2.evaluate(reg, X_train[features2], y_train, X_valid[features2], y_valid, pred, price_lambda)

multiple linear regression 2
Mean Squared Error 34944367428.43664
Root Mean Squared Error 186934.12591
Root Mean Squared Log Error 0.25884
R squared training 0.781
R squared testing 0.744
Adjusted-R squared training 0.781
Adjusted-R squared testing 0.744
intercept [1.48311087]
coefficient [[ 1.51482623e-02  3.41215946e-03  3.78452993e-03  2.31928488e-03
   7.04038947e-03 -1.43355076e-04 -2.82001549e-03  5.83690344e-02
  -2.81136532e-02 -1.12521301e+01]]


#### k-násobná krížová validácia pri viacnásobnej lineárnej regresii

In [12]:
reg=linear_model.LinearRegression()
k = 5

X_try = pd.concat([X_train[features2], X_valid[features2]])
y_try = np.concatenate((np.array(y_train).reshape(-1,1), np.array(y_valid).reshape(-1,1)), axis=0)

scores = cross_val_score(reg,  X_try[features2], np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())
k = 10
scores = cross_val_score(reg,  X_try[features2], np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())

Cross-validated scores: 0.7717217788661574
Cross-validated scores: 0.7718156434452166


### Viacnásobná lineárna regresia - part3

In [13]:
features3 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)
reg=linear_model.LinearRegression()
reg.fit(X_train[features3],y_train)

pred=reg.predict(X_valid[features3])

print('multiple linear regression 3')
metrics2.evaluate(reg, X_train[features3], y_train, X_valid[features3], y_valid, pred, price_lambda)

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436
multiple linear regression 3
Mean Squared Error 32486651893.39338
Root Mean Squared Error 180240.53898
Root Mean Squared Log Error 0.25336
R squared training 0.793
R squared testing 0.754
Adjusted-R squared training 0.793
Adjusted-R squared testing 0.753
intercept [0.41032586]
coefficient [[-1.25747645e-03  1.91255673e-03  1.36418094e-02 -9.06972876e-03
   2.82037240e-03  9.64036987e-03  2.72934373e-03  2.28954703e-03
   6.11502524e-03 -1.41317852e-04  2.31738952e-03 -2.60579557e-03
   9.45836431e-04 -1.05520017e-03 -9.45611848e-04  1.39617770e-03
   4.85389928e-04  6.01748108e-02 -7.75492425e-03  7.45740712e-03
  -1.89647156e-02 -1.07826081e+01]]


### Polynomiálna regresia 2. stupňa 

In [14]:
features1 = feature_selection2.select_features_SFS(X_train, y_train, linear_model.LinearRegression)

polyfeat=PolynomialFeatures(degree=2)
xtrain_poly=polyfeat.fit_transform(X_train[features1])
xvalid_poly=polyfeat.fit_transform(X_valid[features1])

np_y_train = np.array(y_train).reshape(-1,1)
np_y_valid = np.array(y_valid).reshape(-1,1)

poly=linear_model.LinearRegression()
poly.fit(xtrain_poly,np_y_train)
polypred=poly.predict(xvalid_poly)

print('Polynomial regresion - grade 2')

mean_squared_error=metrics.mean_squared_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))

r2_train = poly.score(xtrain_poly, np_y_train)
r2_test = poly.score(xvalid_poly, np_y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train[features1].shape[0] - 1)) / (X_train[features1].shape[0] - X_train[features1].shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid[features1].shape[0] - 1)) / (X_valid[features1].shape[0] - X_valid[features1].shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 5))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),5))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),5))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Polynomial regresion - grade 2
Mean Squared Error 25599854129.10427
Root Mean Squared Error 159999.54415
Root Mean Squared Log Error 0.23187
R squared training 0.828
R squared testing 0.789
Adjusted-R squared training 0.828
Adjusted-R squared testing 0.789


### Polynomiálna regresia 3. stupňa

In [15]:
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)

polyfeat=PolynomialFeatures(degree=3)
xtrain_poly=polyfeat.fit_transform(X_train[features1])
xvalid_poly=polyfeat.fit_transform(X_valid[features1])
np_y_train = np.array(y_train).reshape(-1,1)
np_y_valid = np.array(y_valid).reshape(-1,1)

poly=linear_model.LinearRegression()
poly.fit(xtrain_poly,np_y_train)
polypred=poly.predict(xvalid_poly)

print('Polynomial regresion - grade 3')
mean_squared_error=metrics.mean_squared_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))

r2_train = poly.score(xtrain_poly, np_y_train)
r2_test = poly.score(xvalid_poly, np_y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train[features1].shape[0] - 1)) / (X_train[features1].shape[0] - X_train[features1].shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid[features1].shape[0] - 1)) / (X_valid[features1].shape[0] - X_valid[features1].shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 5))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),5))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),5))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436
Polynomial regresion - grade 3
Mean Squared Error 236372347755.9625
Root Mean Squared Error 486181.39388
Root Mean Squared Log Error 0.21942
R squared training 0.916
R squared testing 0.828
Adjusted-R squared training 0.916
Adjusted-R squared testing 0.827


In [16]:
features1

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'yr_built',
 'zipcode_1',
 'zipcode_2',
 'zipcode_3',
 'zipcode_4',
 'zipcode_5',
 'zipcode_6',
 'zipcode_7',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'price_per_sqft']

### Regresný rozhodovací strom

In [17]:
regr_1 = DecisionTreeRegressor(max_depth=8)
regr_1.fit(X_train, y_train)

y_1 = regr_1.predict(X_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_1, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_1, price_lambda))

r2_train = regr_1.score(X_train, y_train)
r2_test = regr_1.score(X_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Mean Squared Error 23452266212.47
Root Mean Squared Error 153141.33
Root Mean Squared Log Error 0.23
R squared training 0.861
R squared testing 0.802
Adjusted-R squared training 0.861
Adjusted-R squared testing 0.801


### Regresný rozhodovací strom - časť 2

In [30]:
regr_1 = DecisionTreeRegressor(max_depth=8)
regr_1.fit(X_train, y_train)

y_1 = regr_1.predict(X_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_1, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_1, price_lambda))

r2_train = regr_1.score(X_train, y_train)
r2_test = regr_1.score(X_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Mean Squared Error 23462962763.31
Root Mean Squared Error 153176.25
Root Mean Squared Log Error 0.23
R squared training 0.861
R squared testing 0.802
Adjusted-R squared training 0.861
Adjusted-R squared testing 0.801


### Náhodný les

In [33]:
reg_rf = RandomForestRegressor(n_estimators=100, max_depth=8)
reg_rf.fit(X_train, y_train)

y_hat = reg_rf.predict(X_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

r2_train = reg_rf.score(X_train, y_train)
r2_test = reg_rf.score(X_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Mean Squared Error 19663274270.47
Root Mean Squared Error 140225.8
Root Mean Squared Log Error 0.2
R squared training 0.893
R squared testing 0.843
Adjusted-R squared training 0.893
Adjusted-R squared testing 0.842


### SVR

In [84]:
# features1 = ["grade","sqft_living15"]
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)

# clf = SVR(C=1.0, epsilon=0.1, kernel='linear', gamma='auto')
clf = LinearSVR(tol=1e-4, loss='squared_epsilon_insensitive')
X_my_train = X_train[features1]#.values.reshape(-1,1)
X_my_valid = X_valid[features1]#.values.reshape(-1,1)
# clf = SVR(C=1.0, epsilon=0.1)
clf.fit(X_my_train, y_train)

y_hat = clf.predict(X_my_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

r2_train = clf.score(X_my_train, y_train)
r2_test = clf.score(X_my_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_my_train.shape[0] - 1)) / (X_my_train.shape[0] - X_my_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_my_valid.shape[0] - 1)) / (X_my_valid.shape[0] - X_my_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))


# plt.scatter(X_my_valid, y_valid, color = 'magenta')
# plt.plot(X_my_valid, clf.predict(X_my_valid), color = 'green')
# plt.title('Truth or Bluff (Support Vector Regression Model)')
# plt.xlabel('Grade')
# plt.ylabel('Price')
# plt.show()

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Squared Error 48171779383.76
Root Mean Squared Error 219480.7
Root Mean Squared Log Error 0.31
R squared training 0.652
R squared testing 0.627
Adjusted-R squared training 0.652
Adjusted-R squared testing 0.625


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### SGD + Nystroem

In [164]:
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)

clf = SGDRegressor(tol=1e-3, penalty='l1', shuffle=False)
feature_map_nystroem = Nystroem(gamma=.2, random_state=1, n_components=300)

data_transformed_train = feature_map_nystroem.fit_transform(X_train[features1])
data_transformed_valid = feature_map_nystroem.transform(X_valid[features1])

clf.fit(data_transformed_train, np.ravel(y_train))
y_hat = clf.predict(data_transformed_valid)

mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

r2_train = clf.score(data_transformed_train, y_train)
r2_test = clf.score(data_transformed_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train[features1].shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid[features1].shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436
Mean Squared Error 120493650338.01
Root Mean Squared Error 347121.95
Root Mean Squared Log Error 0.56
R squared training -0.221
R squared testing -0.192
Adjusted-R squared training -0.223
Adjusted-R squared testing -0.198


### SGD + RandomSearch

In [185]:
def objective(hyperparameters, iteration, features1):

    clf = SGDRegressor(**hyperparameters)
    feature_map_nystroem = Nystroem(gamma=.2, random_state=1, n_components=300)
    data_transformed_train = feature_map_nystroem.fit_transform(X_train[features1])
    data_transformed_valid = feature_map_nystroem.transform(X_valid[features1])

    clf.fit(data_transformed_train, np.ravel(y_train))

    y_hat = clf.predict(data_transformed_valid)
    
    mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
    rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

    r2_train = clf.score(data_transformed_train, y_train)
    r2_test = clf.score(data_transformed_valid, y_valid)
    adj_r2_train = 1 - (((1 - r2_train) * (X_my_train.shape[0] - 1)) / (X_my_train.shape[0] - X_my_train[features1].shape[1] - 1))
    adj_r2_test = 1 - (((1 - r2_test) * (X_my_valid.shape[0] - 1)) / (X_my_valid.shape[0] - X_my_valid[features1].shape[1] - 1))
    
    score = {
        "MSE": round(mean_squared_error, 2),
        "RMSE": round(np.sqrt(mean_squared_error),2),
        "RMSLE": round(np.sqrt(rmlse),2),
        "R2_train": round(r2_train,3),
        "R2_test": round(r2_test,3),
        "Adj_R2_train": round(adj_r2_train,3),
        "Adj_R2_test": round(adj_r2_test,3)
    }
    
    return [score, hyperparameters]


def random_search(param_grid, max_iterations, features):
    
    df_results = pd.DataFrame(columns = ['MSE', 'RMSE', 'RMSLE', 'R2_train', 'R2_test', 'Adj_R2_train', 'Adj_R2_test', 'params'],
                                  index = list(range(max_iterations)))
    
    for i in range(max_iterations):
        hyperparameters = {k: random.choice(v) for k, v in param_grid.items()}
        
        eval_results = objective(hyperparameters, i, features)
        for k,v in eval_results[0].items():
            df_results[k][i] = v
        df_results['params'][i] = eval_results[1]
    
    # Sort with best score on top
    df_results.sort_values('R2_test', ascending = False, inplace = True)
    df_results.reset_index(inplace=True)
    
    return df_results

param_grid = {
    'alpha': np.random.uniform(1e-5,1e-3,100),
#     'average': [True,False],
#     'early_stopping': [True],
#     'epsilon': [0.1],
#     'eta0': np.random.uniform(0.001,0.1,100),
#     'fit_intercept': [True],
    'l1_ratio': np.random.uniform(0.1,0.9,100),
    'learning_rate': ['constant','optimal', 'adaptive','invscaling'],
    'loss': ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'max_iter': [10000],
    'n_iter_no_change': [5],
    'penalty': ['l1','l2','elasticnet'],
    'power_t': np.random.uniform(0.15,0.4,100),
    'shuffle': [False],
    "tol":np.random.uniform(1e-4, 1e-2, 100),
    'validation_fraction': np.arange(0.1, 0.3, 0.05),
}

random.seed(50)
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)
df_results2 = random_search(param_grid, 50, features1)
df_results2

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:1185: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').